In [35]:
%run imports.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download food nutrient data

In [56]:
driver = webdriver.Firefox()

In [57]:
def waitForLoad(selector): 
    Wait = WebDriverWait(driver, 10)       
    Wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

def get_food_groups():
    # Main page.
    url = 'http://nevo-online.rivm.nl/Default.aspx'
    driver.get(url)
    driver.find_element_by_css_selector('#ctl00_ContentBody_btZoekProducten').click()
    # Search by food group.
    next_id = '#ctl00_ContentBody_btZoekOpProductGroep'
    waitForLoad(next_id)
    driver.find_element_by_css_selector(next_id).click()
    # Extract table.
    next_id = '#ctl00_ContentBody_gvProductGroepen'
    waitForLoad(next_id)
    return pd.read_html(driver.page_source, header=0)[0].iloc[:, 1:]

food_groups_df = get_food_groups()
food_groups_df

,Voedingsmiddelengroep
0,Aardappelen
1,Alcoholische en niet-alcoholische dranken
2,Brood
3,Diversen
4,Eieren
5,Fruit
6,Gebak en koek
7,Graanproducten en bindmiddelen
8,Groenten
9,Hartig broodbeleg


In [ ]:
nutrients_df = None
for i, food_group in tqdm_notebook(food_groups_df.iloc[:, 0].iteritems(), unit='food group', total=len(food_groups_df)):
    # Main page.
    url = 'http://nevo-online.rivm.nl/Default.aspx'
    driver.get(url)
    driver.find_element_by_css_selector('#ctl00_ContentBody_btZoekProducten').click()
    # Search by food group.
    next_id = '#ctl00_ContentBody_btZoekOpProductGroep'
    waitForLoad(next_id)
    driver.find_element_by_css_selector(next_id).click()
    # Select food group i.
    next_id = '#ctl00_ContentBody_gvProductGroepen'
    waitForLoad(next_id)
    driver.execute_script(f"__doPostBack('ctl00$ContentBody$gvProductGroepen','Select${i}')")
    # Select all foods in this group.
    next_id = '#ctl00_ContentBody_gvGevondenProducten_ctl01_chkAll'
    waitForLoad(next_id)
    driver.find_element_by_css_selector(next_id).click()
    time.sleep(1)
    # Show selected foods.
    driver.execute_script("__doPostBack('ctl00$ContentBody$btToonNutrienten','')")
    #next_id = '#ctl00_ContentBody_gvTest'
    #waitForLoad(next_id)
    time.sleep(8)
    # Load and extract each tab's table.
    food_group_df = pd.read_html(
        driver.page_source, header=0, index_col=1)[0].iloc[:, 1:]
    food_group_df['Voedselgroep'] = food_group
    for i in range(2, 12):
        next_tab = re.sub(r'tabid=\d+', f'tabid={i}', driver.current_url)
        driver.get(next_tab)
        tab_df = pd.read_html(driver.page_source, header=0, index_col=1)[0].iloc[:, 1:]
        food_group_df = food_group_df.combine_first(tab_df)
    if nutrients_df is None:
        nutrients_df = food_group_df
    else:
        nutrients_df = nutrients_df.combine_first(food_group_df)

In [64]:
# Reorder the columns and store to disk.
cols_front = ['Voedselgroep', 'Voedingsmiddel', 'Food item']
cols_back = sorted([col for col in nutrients_df.columns if col not in cols_front])
nutrients_df = nutrients_df[cols_front + cols_back].sort_index()
nutrients_df.to_msgpack('nutrients-nl.msg', compress='zlib')

In [65]:
nutrients_df[[col for col in nutrients_df.columns if ':' not in col]].head(20)

,Voedselgroep,Voedingsmiddel,Food item,25-hydroxy-vitamine D (ug),Alcohol totaal (g),Alfa-caroteen (ug),Alfa-tocoferol (mg),As (g),Beta-caroteen (ug),Beta-cryptoxanthine (ug),Beta-tocoferol (mg),Calcium (mg),Cholecalciferol - vit D3 (ug),Cholesterol (mg),Delta-tocoferol (mg),Eiwit dierlijk (g),Eiwit plantaardig (g),Eiwit totaal (g),Energie (kJ),Energie (kcal),Eovz cis rest (%),Eovz trans rest (%),Folaat (ug),Folaat equivalenten (ug),Foliumzuur (ug),Fosfor (mg),Gamma-tocoferol (mg),IJzer haem (mg),IJzer non haem (mg),IJzer totaal (mg),Jodium (ug),Kalium (mg),Koolhydraten totaal (g),Koper (mg),Luteine (ug),Lycopeen (ug),Magnesium (mg),Mono en disacchariden totaal (g),Movz rest c+t (%),Natrium (mg),Nicotinezuur (mg),Niet-geidentificeerde vetzuren (%),Organische zuren totaal (g),Polyolen totaal (g),Polysacchariden totaal (g),Retinol (ug),Retinol activiteit equiv.(RAE) (ug),Retinol equivalenten (RE) (ug),Selenium (ug),Som van de vetzuren (g),Stikstof totaal (g),Verz vetz rest (%),Vet totaal (g),Vetzuren enkelv onverz cis (g),Vetzuren meerv onverz (g),Vetzuren n-3 meerv onverz cis (g),Vetzuren n-6 meerv onverz cis (g),Vetzuren totaal trans (g),Vetzuren totaal verzadigd (g),Vitamine B1 (mg),Vitamine B12 (ug),Vitamine B2 (mg),Vitamine B6 (mg),Vitamine C (mg),Vitamine D totaal (ug),Vitamine E totaal (mg),Vitamine K totaal (ug),Vitamine K1 (ug),Vitamine K2 (ug),Voedingsvezel totaal (g),Water (g),Zeaxanthine (ug),Zink (mg)
NEVO-code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Aardappelen,Aardappelen rauw,Potatoes raw,0,0,0,--,--,5,3,--,6,0,0,--,0,2,2,371.0,88.0,0,0,23,23,0,60,--,0,0.5,0.5,2.5,450,19,0.09,100,0,22,1,0,2,--,0,--,0,18,0,1,1,1,0,--,0,0,0,0,0,0,0,0,0.12,0,0.04,0.3,14,0,--,--,--,--,1.8,77,16,0.37
2,Aardappelen,Aardappelen nieuwe rauw,Potatoes new raw,0,0,0,--,--,3,0,--,6,0,0,--,0,2,2,371.0,88.0,0,0,23,23,0,60,--,0,0.5,0.5,2.5,450,19,0.09,13,0,14,1,0,2,1.2,0,--,0,18,0,0,0,1,0,--,0,0,0,0,0,0,0,0,0.12,0,0.04,0.3,25,0,0.1,--,--,--,1.8,77,4,0.2
3,Aardappelen,Aardappelen oude rauw,Potatoes old raw,0,0,spoor,--,--,8,--,--,6,0,0,--,0,2,2,371.0,88.0,0,0,23,23,0,60,--,0,0.5,0.5,2.5,450,19,0.08,60,0,17,1,0,2,1.2,0,--,0,18,0,1,1,1,0,--,0,0,0,0,0,0,0,0,0.12,0,0.04,0.22,5,0,0.1,--,--,--,1.8,77,--,0.3
4,Graanproducten en bindmiddelen,Pasta witte rauw,Pasta white raw,--,0,--,--,--,0,--,--,15,--,0,--,0,12.3,12.3,1511.0,356.0,--,--,25,25,0,125,--,0,2,2,0.6,100,72,0.3,--,--,53,1,--,2,2,3.4,NaN,0,71,0,0,0,20,1,--,--,1.5,0.2,0.6,0,0.6,0,0.2,0.1,0,0.06,0.08,0,0,spoor,0.2,0.2,NaN,3,10,--,1.5
5,Graanproducten en bindmiddelen,Rijst witte rauw,Rice white raw,--,0,--,--,--,0,--,--,10,--,0,--,0,7,7,1492.0,352.0,0,--,20,20,0,100,--,0,0.4,0.4,2.2,100,78,0.37,--,--,13,0,spoor,2,1,--,NaN,0,78,0,0,0,13,0.9,--,--,1,0.3,0.4,0,0.4,0,0.2,0.04,0,0.03,0.12,0,0,0.1,0.1,0.1,NaN,1.3,13,--,1.8
7,Groenten,Andijvie rauw,Endive raw,--,0,3,0.3,0.6,905,0,0,54,--,0,0,0,1.3,1.3,71.0,17.0,0,0,35.2,35.2,0,36,0.4,0,1.2,1.2,6.4,253,1.2,0.05,1516,16,11,1.2,0,22,0.4,0,--,0,0,0,75,151,1,0.3,--,0,0.4,0,0.2,0,0.1,0,0.1,0.05,0,0.06,0.057,1,0,0.3,231,231,--,1.8,95.3,4,0.55
8,Groenten,Andijvie gekookt,Endive boiled,--,0,8,0.6,0.7,2240,39,0,60,--,0,0,0,1.4,1.4,95.0,23.0,0,0,45,45,0,25,1.2,0,0.5,0.5,6.4,163,2.4,0.06,2926,26,9,1.6,0,5,0.5,0,--,0,0.8,0,188,378,0,0.3,--,0,0.5,0,0.2,0,0.2,0,0.1,0.05,0,0.06,0.081,1,0,0.8,337,337,--,1.7,93.5,6,0.27
9,Groenten,Asperge rauw,Asparagus raw,--,0,1,--,--,1,0,--,20,--,0,--,0,1,1,82.0,19.0,0,0,56,56,0,60,--,0,1,1,0.2,200,3,0.08,25,0,20,1,0,1,0.6,0,--,0,2,0,0,0,1,0,--,0,0,0,0,0,0,0,0,0.1,0,0.07,0.03,30,0,1.2,41.6,41.6,--,1.7,94,0,0.5
10,Groenten,Aubergine rauw,Aubergine raw,--,0,60,--,--,40,0,--,10,--,0,--,0,1,1,84.0,20.0,0,0,18,18,0,12,--,0,0.5,0.5,1,200,3,0.01,--,--,11,3,0,2,0.6,0,--,0,0,0,6,12,0,0,--,0,0,0,0,0,0,0,0,0.04,0,0.05,0.07,10,0,0,6.1,6.1,--,2,93,--,0.2
